# Process FATHOM flooding

- current pluvial and fluvial - read and rewrite with zeros over nodata/999 values

- future pluvial and fluvial: apply change factors

In [ ]:
import itertools
import os
import pathlib
import re
from glob import glob

import numpy as np
import pandas as pd
import rasterio
from tqdm.notebook import tqdm


In [ ]:
# Change to incoming_data, storm surge data folder
os.chdir("../../incoming_data/FATHOM Flood")
os.getcwd()


In [ ]:
out_dir = "../../processed_data/hazards/fathom_pluvial_fluvial/"


In [ ]:
!mkdir -p {out_dir}


In [ ]:
# Set of return periods in data, plus artificial lower/upper bound
RPS = np.array([0, 5, 10, 20, 50, 75, 100, 200, 250, 500, 1000, 1e6])
# change factors available for [5, 10, 50, 100]


def read_rp_map(fname):
    """Read flood map, use all cells with any depth > 0 as potential exposure
    points
    """
    with rasterio.open(fname) as dataset:
        data = dataset.read(1)
        data[data > 990] = 0
        data[data < 0] = 0
        data[data == dataset.nodata] = 0
        np.nan_to_num(data, copy=False)

    return data


def read_transform(fname):
    with rasterio.open(fname) as dataset:
        crs = dataset.crs
        ncols = dataset.width
        nrows = dataset.height
        transform = dataset.transform
        nodata = dataset.nodata
    return crs, ncols, nrows, transform, nodata


def read_rp_maps(fnames, rps):
    rp_data = {}
    # Read the rest
    for fname, rp in zip(fnames, rps):
        rp_data[rp] = read_rp_map(fname)

    return rp_data


def save_to_tif(data, fname, nrows, ncols, crs, transform):
    with rasterio.open(
        fname,
        "w",
        driver="GTiff",
        height=nrows,
        width=ncols,
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform,
        compress="lzw",
    ) as dataset:
        dataset.write(data, 1)


In [ ]:
for f in tqdm(list(sorted(glob("**/*.tif", recursive=True)))):
    f = pathlib.Path(f)

    country_name, hazard = f.parent.parts

    match country_name:
        case "Dominica":
            isoa3 = "dma"
        case "St Vincent and the Grenadines":
            isoa3 = "vct"
        case "St Lucia":
            isoa3 = "lca"
        case "Grenada":
            isoa3 = "grd"

    hazard_short, rp = re.match(r"(\w+)_1in(\d+).tif", f.name).group(1, 2)

    epoch = "2010"
    rcp = "baseline"
    out_fname = f"{hazard}__epoch_{epoch}__rcp_{rcp}__precipitation-factor_none__stat_none__rp_{rp}__isoa3_{isoa3}.tif"

    with rasterio.open(f) as dataset:
        crs = dataset.crs
        ncols = dataset.width
        nrows = dataset.height
        transform = dataset.transform
        nodata = dataset.nodata
    data = read_rp_map(f)
    save_to_tif(data, os.path.join(out_dir, out_fname), nrows, ncols, crs, transform)


## Apply change factors

In [ ]:
statistics = ['p10', 'median', 'p90']  # median, p10, p90
ssps = ["ssp126", "ssp245", "ssp585"]
epochs = [2030, 2050]
predict_rps = [
    5,
    10,
    50,
    100,
]  # skip 25 as no correspondance to data, skip 20 as no equivalent in storm surge
variables = ["rx1day", "rx5day", "rxmonth"]

change_factors = pd.read_csv("../wbcckp/summary.csv")
change_factors["rp"] = change_factors.calculation.str.extract(r"(\d+)").astype(int)
change_factors["rp_future"] = change_factors.rp * change_factors.value


def rename_epoch(epoch):
    match epoch:
        case "2010-2039":
            epoch = 2030
        case "2035-2064":
            epoch = 2050
        case "2060-2089":
            epoch = 2070
        case "2070-2099":
            epoch = 2080
    return epoch


change_factors.epoch = change_factors.epoch.apply(rename_epoch)

# copy full DataFrame for later exploration
cf = change_factors.copy()

# for col in change_factors.columns:
#     print(col, "\n", np.sort(change_factors[col].unique()), "\n")

change_factors = change_factors.query(
    f"ssp in {ssps} and epoch in {epochs} and rp in {predict_rps} and statistic in {statistics}"
)


In [ ]:
cf.query(
    'ssp == "ssp245" and isoa3 == "VCT" and rp == 100 and epoch == 2050'
)#[['variable', 'statistic', 'rp', 'rp_future']]


In [ ]:
change_factors.query(
    'variable == "rx1day" and ssp == "ssp245" and isoa3 == "VCT" '
).sort_values(by=["epoch", "rp"])


In [ ]:
rp_future_lookup = change_factors.set_index(
    ["isoa3", "statistic", "variable", "ssp", "epoch", "rp"]
)["rp_future"].sort_index()
rp_future_lookup["VCT", "median", "rx1day", "ssp126", 2050]


In [ ]:
def interpolate_rp_factor(rp, rp_l, rp_u):
    return (np.log(rp) - np.log(rp_l)) / (np.log(rp_u) - np.log(rp_l))


def interpolate_depth(depth_l, depth_u, rp_factor):
    return depth_l + ((depth_u - depth_l) * rp_factor)


for isoa3, hazard in tqdm(list(itertools.product(
    ("vct", "dma", "grd", "lca"), ("fluvial_undefended", "fluvial_defended", "pluvial")
))):
    if "fluvial" in hazard and isoa3 == "vct":
        # no data
        continue
    # Read baseline depths
    baseline_rps = (5, 10, 20, 50, 75, 100, 200, 250, 500, 1000)
    fnames = [
        f"../../processed_data/hazards/fathom_pluvial_fluvial/{hazard}__epoch_2010__rcp_baseline__precipitation-factor_none__stat_none__rp_{baseline_rp}__isoa3_{isoa3}.tif"
        for baseline_rp in baseline_rps
    ]
    rp_depths = read_rp_maps(fnames, baseline_rps)
    rp_depths[0] = np.zeros_like(rp_depths[5])
    crs, ncols, nrows, transform, nodata = read_transform(fnames[0])

    # Predict
    for stat, var, ssp, epoch, rp in itertools.product(statistics, variables, ssps, epochs, predict_rps):
        rp_future = rp_future_lookup[isoa3.upper(), stat, var, ssp, epoch, rp]
        #print(isoa3, stat, var, ssp, epoch, rp, rp_future)
        out_fname = (
            "../../processed_data/hazards/fathom_pluvial_fluvial/"
            f"{hazard}__epoch_{epoch}__rcp_{ssp}__precipitation-factor_{var}__stat_{stat}__rp_{rp}__isoa3_{isoa3}.tif"
        )
        rp_u_idx = np.searchsorted(RPS, rp_future, side="left")
        rp_l, rp_u = RPS[rp_u_idx - 1], RPS[rp_u_idx]
        rp_factor = interpolate_rp_factor(rp_future, rp_l, rp_u)
        depth_l = rp_depths[rp_l]
        depth_u = rp_depths[rp_u]
        depth_future = interpolate_depth(depth_l, depth_u, rp_factor)

        save_to_tif(depth_future, out_fname, nrows, ncols, crs, transform)


In [ ]:
flooded_volume = []

for isoa3, hazard in tqdm(list(itertools.product(
    ("vct", "dma", "grd", "lca"), ("fluvial_undefended", "fluvial_defended", "pluvial")
))):
    if "fluvial" in hazard and isoa3 == "vct":
        # no data
        continue
    baseline_rps = (5, 10, 20, 50, 75, 100, 200, 250, 500, 1000)
    for rp in baseline_rps:
        fname = (
            "../../processed_data/hazards/fathom_pluvial_fluvial/"
            f"{hazard}__epoch_2010__rcp_baseline__precipitation-factor_none__stat_none__rp_{rp}__isoa3_{isoa3}.tif"
        )
        data = read_rp_map(fname)
        flooded_volume.append({
            "hazard": hazard,
            "epoch": 2010,
            "rcp": "baseline",
            "precipitation-factor": "none",
            "stat": "none",
            "rp": rp,
            "isoa3": isoa3,
            "pixel_nonzero_count": np.count_nonzero(data),
            "pixel_sum": data.sum(),
        })

    for stat, var, ssp, epoch, rp in itertools.product(statistics, variables, ssps, epochs, predict_rps):
        fname = (
            "../../processed_data/hazards/fathom_pluvial_fluvial/"
            f"{hazard}__epoch_{epoch}__rcp_{ssp}__precipitation-factor_{var}__stat_{stat}__rp_{rp}__isoa3_{isoa3}.tif"
        )
        data = read_rp_map(fname).flatten()

        flooded_volume.append({
            "hazard": hazard,
            "epoch": epoch,
            "rcp": ssp,
            "precipitation-factor": var,
            "stat": stat,
            "rp": rp,
            "isoa3": isoa3,
            "pixel_nonzero_count": np.count_nonzero(data),
            "pixel_sum": data.sum(),
        })

flooded_volume = pd.DataFrame(flooded_volume)


In [ ]:
flooded_volume \
    .to_csv("../../processed_data/hazards/fathom_pluvial_fluvial/depth_comparison.csv", index=False)


In [ ]:
pd.DataFrame(rp_future_lookup) \
    .rename(columns={"rp_future": "rp_present_equivalent"}) \
    .to_csv("../../processed_data/hazards/fathom_pluvial_fluvial/rp_comparison.csv")


## Explore change factor values

In [ ]:
cf.epoch = pd.to_datetime(cf.epoch, format="%Y")


In [ ]:
for isoa3 in ["VCT", "DMA", "GRD", "LCA"]:
    df = cf.query(
        f'ssp=="ssp245" and isoa3 == "{isoa3}" and calculation == "changefactorfaep100yr" and statistic == "p90"'
    ).pivot(index="epoch", columns="variable", values="value").plot()


In [ ]:
for variable in ["rx1day", "rx5day", "rxmonth"]:
    cf.query(
        f'isoa3 == "VCT" and calculation == "changefactorfaep100yr" and variable == "{variable}" and statistic == "p90"'
    ).pivot(index="epoch", columns="ssp", values="value").plot()


In [ ]:
for stat in ("p90", "median", "p10"):
    cf.query(f'isoa3 == "VCT" and ssp == "ssp245" and variable == "rx5day" and statistic == "{stat}"').pivot(
        index="epoch", columns="rp", values="value"
    ).plot()
